In [1]:
import jax
import jax.numpy as jnp
from crystalformer.src.utils import GLXYZAW_from_file
from crystalformer.src.elements import element_dict
from crystalformer.src.formula import find_composition_vector, formula_string
from crystalformer.src.wyckoff import mult_table

ERROR:2025-09-23 21:30:28,418:jax._src.xla_bridge:487: Jax plugin configuration error: Exception when calling jax_plugins.xla_cuda12.initialize()
Traceback (most recent call last):
  File "/home/user_wanglei/.local/lib/python3.13/site-packages/jax/_src/xla_bridge.py", line 485, in discover_pjrt_plugins
    plugin_module.initialize()
    ~~~~~~~~~~~~~~~~~~~~~~~~^^
  File "/home/user_wanglei/.local/lib/python3.13/site-packages/jax_plugins/xla_cuda12/__init__.py", line 328, in initialize
    _check_cuda_versions(raise_on_first_error=True)
    ~~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/user_wanglei/.local/lib/python3.13/site-packages/jax_plugins/xla_cuda12/__init__.py", line 285, in _check_cuda_versions
    local_device_count = cuda_versions.cuda_device_count()
RuntimeError: jaxlib/cuda/versions_helpers.cc:113: operation cuInit(0) failed: Unknown CUDA error 303; cuGetErrorName failed. This probably means that JAX was unable to load the CUDA libraries.


In [2]:
alex20_folder = '/opt/data/bcmdata/ZONES/data/PROJECTS/datafile/PRIVATE/zdcao/crystal_gpt/dataset/alex/PBE/alex20/'

train_path = alex20_folder+'/train.lmdb'
valid_path = alex20_folder+'/val.lmdb'
test_path = alex20_folder+'/test.lmdb'


In [3]:
atom_types = 119 
wyck_types = 28 
n_max = 21

train_dataset = GLXYZAW_from_file(train_path, atom_types, wyck_types, n_max)
valid_dataset = GLXYZAW_from_file(valid_path, atom_types, wyck_types, n_max)
test_dataset = GLXYZAW_from_file(test_path, atom_types, wyck_types, n_max)





G: (1071694,)
L: (1071694, 6)
XYZ: (1071694, 21, 3)
A: (1071694, 21)
W: (1071694, 21)
G: (133962,)
L: (133962, 6)
XYZ: (133962, 21, 3)
A: (133962, 21)
W: (133962, 21)
G: (133962,)
L: (133962, 6)
XYZ: (133962, 21, 3)
A: (133962, 21)
W: (133962, 21)


In [4]:
G, L, XYZ, A, W = train_dataset

In [5]:
def find_element(x, elements):
    allowed_values = [0] + [element_dict[e] for e in elements]
    allowed = jnp.asarray(allowed_values)

    # eq[i, j, k] = (x[i, j] == allowed[k])
    eq = (x[:, :, None] == allowed[None, None, :])

    # (1) only allowed values: for each element, it's equal to at least one allowed
    only_allowed = jnp.all(jnp.any(eq, axis=2), axis=1)   # shape (n,)

    # (2) all allowed present: for each allowed value k, it appears in the row at least once
    all_present = jnp.all(jnp.any(eq, axis=1), axis=1)    # shape (n,)

    return jnp.where(only_allowed & all_present)[0]

In [13]:
idx = find_element(A, ['La', 'Ni', 'O'])


In [14]:
A[idx]

Array([[28,  8,  8, 28,  8, 57, 57,  8,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0],
       [28,  8,  8, 57,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0],
       [28,  8, 57,  8,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0],
       [28,  8, 57,  8,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0],
       [28,  8,  8, 57,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0],
       [28, 57,  8,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0]], dtype=int32)

In [15]:
G[idx]

Array([139, 139,  62, 127,  64, 221], dtype=int32)

In [16]:
A[idx]

Array([[28,  8,  8, 28,  8, 57, 57,  8,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0],
       [28,  8,  8, 57,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0],
       [28,  8, 57,  8,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0],
       [28,  8, 57,  8,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0],
       [28,  8,  8, 57,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0],
       [28, 57,  8,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0]], dtype=int32)

In [17]:
W[idx]

Array([[1, 3, 5, 5, 5, 5, 5, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 3, 5, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [2, 3, 3, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 2, 3, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [2, 5, 6, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 2, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],      dtype=int32)

In [18]:
L[idx]

Array([[1.1957334, 1.1957334, 8.646317 , 1.5707964, 1.5707964, 1.5707964],
       [1.6265556, 1.6265556, 5.1895986, 1.5707964, 1.5707964, 1.5707964],
       [2.0151935, 2.8626564, 2.0390313, 1.5707964, 1.5707964, 1.5707964],
       [2.5260882, 2.5260882, 1.8192531, 1.5707964, 1.5707964, 1.5707964],
       [1.8176199, 4.0900226, 1.8760359, 1.5707964, 1.5707964, 1.5707964],
       [2.2546456, 2.2546456, 2.2546456, 1.5707964, 1.5707964, 1.5707964]],      dtype=float32)

In [19]:
@jax.vmap
def lookup(g, w):
    return mult_table[g-1, w]
M = lookup(G[idx], W[idx]) # (batchsize, n_max)


In [20]:
composition = jax.vmap(find_composition_vector)(A[idx], M)

In [21]:
for c in composition:
    print (formula_string(c))

Ni3La4O10
NiLa2O4
NiLaO3
NiLaO3
NiLa2O4
NiLaO3
